# 모델 깊게 짜보기 

In [1]:
# 필요한 라이브러리를 임포트합니다.
import pandas as pd  # 데이터 조작 및 분석을 위한 라이브러리
import numpy as np  # 수치 계산을 위한 라이브러리
import wfdb  # 심전도 데이터를 읽기 위한 라이브러리
import ast  # 문자열 형태의 파이썬 표현식을 파싱하기 위한 라이브러리
from tqdm import tqdm  # 반복 작업의 진행 상황을 시각적으로 표시하기 위한 라이브러리
import warnings; warnings.filterwarnings('ignore')  # 경고 메시지를 무시하기 위한 설정
from IPython.display import display  # Jupyter 노트북에서 데이터를 깔끔하게 보여주기 위한 함수
from glob import glob # 하위 디렉토리에 파일을 가져와서 다루기위한 라이브러리 
import os # 현재 워킹디렉토리의 경로와 파일을 다루기 위한 라이브러리 
import h5py #.h5파일을 다루기 위한 라이브러리 
import gc #가비치 컬렉터를 다루는 라이브러리
import random


# 데이터 시각화를 위한 라이브러리를 임포트합니다.
import matplotlib.pyplot as plt  # 그래프를 그리기 위한 라이브러리
import seaborn as sns  # matplotlib 기반의 고급 시각화 라이브러리

#머신러닝을 위한 scikit-learn 임포트 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report 
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# 파이토치를 위한 라이브러리 임포트 
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset # 텐서데이터셋
import torch.optim as optim

# 평가 지표 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import roc_curve, auc


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_auc_score, accuracy_score, precision_recall_curve, 
    precision_score, recall_score, f1_score, confusion_matrix, roc_curve, average_precision_score
)

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator
from tqdm import tqdm


import json
import yaml
import random
import wandb
import ast
import optuna

In [3]:
## 모델 시드 고정 
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # 멀티 GPU를 사용하는 경우
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [4]:
## ptb,sph데이터 

# 데이터 프레임 로드
path = './ptb_xl_data/'
df_ptb = pd.read_csv(path + 'ptbxl_database.csv', index_col='ecg_id')
df_sph = pd.read_csv("./sph_data/metadata.csv")

# str (문자열) 코드를 딕셔너리로 변환 
df_ptb.scp_codes = df_ptb.scp_codes.apply(lambda x: ast.literal_eval(x))

# 딕셔너리에서 라벨 추출 ()
df_ptb.scp_codes = df_ptb.scp_codes.apply(lambda x: list(x.keys()))

# 심방세동 라벨 컬럼 생성 (정답 컬럼)
df_ptb['label'] = df_ptb.scp_codes.apply(lambda arr: 1 if 'AFIB' in arr else 0)

# 'AHA_Code' 컬럼의 각 값에 대해 '50'이 포함되어 있는지 확인하고, 'label' 컬럼 생성
def check_contains_50(code):
    # 공백을 없애고, ';' 또는 '+'로 분리
    numbers = code.replace(' ', '').replace('+', ';').split(';')
    # '50'이 포함되어 있는지 확인
    return '50' in numbers
# 'label' 컬럼 생성
df_sph['label'] = df_sph['AHA_Code'].apply(check_contains_50).astype(int)


## 보경님 코드를 이용한 전처리 

import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold


# custom_file 폴더에서 lead1_signals 데이터 로드
lead1_signals = np.load('./custom_file/annie_ptb_xl_lead1.npy')


# 레이블 데이터 로드
labels = df_ptb['label'].values

lead1_signals_float=lead1_signals.astype(np.float32)



# 데이터 정규화
lead1_signals_normalized = (lead1_signals_float - lead1_signals_float.mean()) / (lead1_signals_float.std()+1e-7)


# 데이터셋을 텐서로 변환
X = torch.Tensor(lead1_signals_normalized)
y = torch.Tensor(labels).long()  


# 데이터 분할
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# 채널 수를 1로 추가_GOOD
X_train = X_train.unsqueeze(1)
X_val = X_val.unsqueeze(1)
X_test = X_test.unsqueeze(1)


# DataLoader 생성
train_data = TensorDataset(X_train, y_train)
val_data = TensorDataset(X_val, y_val)
test_data = TensorDataset(X_test, y_test)

batch_size = 32
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)


## 기존 모델 


In [21]:
# 충원님 코드

class Custom1DCNN1(nn.Module):
    def __init__(self):
        super(Custom1DCNN1, self).__init__()

        # Convolutional Blocks
        self.conv1 = nn.Conv1d(1, 32, kernel_size=5)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.maxpool1 = nn.MaxPool1d(2)

        self.conv2 = nn.Conv1d(32, 32, kernel_size=5)
        self.maxpool2 = nn.MaxPool1d(2)

        self.conv3 = nn.Conv1d(32, 64, kernel_size=5)
        self.maxpool3 = nn.MaxPool1d(2)

        self.conv4 = nn.Conv1d(64, 64, kernel_size=5)
        self.maxpool4 = nn.MaxPool1d(2)

        self.conv5 = nn.Conv1d(64, 128, kernel_size=5)
        self.maxpool5 = nn.MaxPool1d(2)

        self.conv6 = nn.Conv1d(128, 128, kernel_size=5)
        self.maxpool6 = nn.MaxPool1d(2)

        self.dropout1 = nn.Dropout(0.5)

        self.conv7 = nn.Conv1d(128, 256, kernel_size=5)
        self.maxpool7 = nn.MaxPool1d(2)

        self.conv8 = nn.Conv1d(256, 256, kernel_size=5)
        self.maxpool8 = nn.MaxPool1d(2)

        self.dropout2 = nn.Dropout(0.5)

        self.conv9 = nn.Conv1d(256, 512, kernel_size=5)
        self.maxpool9 = nn.MaxPool1d(2)

        self.dropout3 = nn.Dropout(0.5)

        self.conv10 = nn.Conv1d(512, 512, kernel_size=5)

        # Fully Connected Blocks
        self.flatten = nn.Flatten()

        self.dense1 = nn.Linear(512, 128)
        self.batch_norm_dense1 = nn.BatchNorm1d(128)  # BatchNorm1d for Dense1
        self.dropout4 = nn.Dropout(0.5)

        self.dense2 = nn.Linear(128, 32)
        self.batch_norm_dense2 = nn.BatchNorm1d(32)  # BatchNorm1d for Dense2

        self.dense3 = nn.Linear(32, 1)
        

    def forward(self, x):
        # Convolutional Blocks
        x = self.maxpool1(self.relu(self.bn1(self.conv1(x))))
        x = self.maxpool2(self.relu(self.conv2(x)))
        x = self.maxpool3(self.relu(self.conv3(x)))
        x = self.maxpool4(self.relu(self.conv4(x)))
        x = self.maxpool5(self.relu(self.conv5(x)))
        x = self.maxpool6(self.relu(self.conv6(x)))
        x = self.dropout1(x)
        x = self.maxpool7(self.relu(self.conv7(x)))
        x = self.maxpool8(self.relu(self.conv8(x)))
        x = self.dropout2(x)
        x = self.maxpool9(self.relu(self.conv9(x)))
        x = self.dropout3(x)
        x = self.conv10(x)

        # Fully Connected Blocks
        x = self.flatten(x)
        x = self.dropout4(self.relu(self.batch_norm_dense1(self.dense1(x))))
        x = self.relu(self.batch_norm_dense2(self.dense2(x)))
        x = self.dense3(x)

        return x

## 좀더 깊게 짠 모델 

In [30]:
class Custom1DCNN2(nn.Module):
    def __init__(self):
        super(Custom1DCNN2, self).__init__()

        # Convolutional Blocks
        self.conv1 = nn.Conv1d(1, 32, kernel_size=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.maxpool1 = nn.MaxPool1d(2)

        self.conv2 = nn.Conv1d(32, 32, kernel_size=2)
        self.maxpool2 = nn.MaxPool1d(2)

        self.conv3 = nn.Conv1d(32, 64, kernel_size=2)
        self.maxpool3 = nn.MaxPool1d(2)

        self.conv4 = nn.Conv1d(64, 64, kernel_size=2)
        self.maxpool4 = nn.MaxPool1d(2)

        self.conv5 = nn.Conv1d(64, 128, kernel_size=2)
        self.maxpool5 = nn.MaxPool1d(2)

        self.conv6 = nn.Conv1d(128, 128, kernel_size=2)
        self.maxpool6 = nn.MaxPool1d(2)

        self.dropout1 = nn.Dropout(0.5)

        self.conv7 = nn.Conv1d(128, 256, kernel_size=2)
        self.maxpool7 = nn.MaxPool1d(2)

        self.conv8 = nn.Conv1d(256, 256, kernel_size=2)
        self.maxpool8 = nn.MaxPool1d(2)

        self.dropout2 = nn.Dropout(0.5)

        self.conv9 = nn.Conv1d(256, 512, kernel_size=2)
        self.maxpool9 = nn.MaxPool1d(2)

        self.dropout3 = nn.Dropout(0.5)

        self.conv10 = nn.Conv1d(512, 512, kernel_size=2)
        self.maxpool10 = nn.MaxPool1d(2)
        
        self.conv11 = nn.Conv1d(512, 1024, kernel_size=2)
        self.maxpool11 = nn.MaxPool1d(2)
        
        self.dropout4 = nn.Dropout(0.5)
        self.conv12 = nn.Conv1d(1024, 1024, kernel_size=1)

        # Fully Connected Blocks
        self.flatten = nn.Flatten()

        self.dense1 = nn.Linear(1024, 512)
        
        self.batch_norm_dense1 = nn.BatchNorm1d(512)  # BatchNorm1d for Dense1
        self.dropout5 = nn.Dropout(0.5)
        
        self.dense2 = nn.Linear(512, 128)
        
        self.batch_norm_dense2 = nn.BatchNorm1d(128)  # BatchNorm1d for Dense1
        self.dropout6 = nn.Dropout(0.5)

        self.dense3 = nn.Linear(128, 32)
        self.batch_norm_dense3 = nn.BatchNorm1d(32)  # BatchNorm1d for Dense2

        self.dense4 = nn.Linear(32, 1)
        

    def forward(self, x):
        # Convolutional Blocks
        x = self.maxpool1(self.relu(self.bn1(self.conv1(x))))
        x = self.maxpool2(self.relu(self.conv2(x)))
        x = self.maxpool3(self.relu(self.conv3(x)))
        x = self.maxpool4(self.relu(self.conv4(x)))
        x = self.maxpool5(self.relu(self.conv5(x)))
        x = self.maxpool6(self.relu(self.conv6(x)))
        x = self.dropout1(x)
        x = self.maxpool7(self.relu(self.conv7(x)))
        x = self.maxpool8(self.relu(self.conv8(x)))
        x = self.dropout2(x)
        x = self.maxpool9(self.relu(self.conv9(x)))
        x = self.dropout3(x)
        x = self.maxpool10(self.relu(self.conv10(x)))
        x = self.maxpool11(self.relu(self.conv11(x)))
        x = self.dropout4(x)
        x = self.conv12(x)

        # Fully Connected Blocks
        x = self.flatten(x)
        x = self.dropout5(self.relu(self.batch_norm_dense1(self.dense1(x))))
        x = self.relu(self.batch_norm_dense2(self.dense2(x)))
        x = self.dropout6(self.relu(self.batch_norm_dense3(self.dense3(x))))
        x = self.dense4(x)

        return x

In [28]:

conv1 = nn.Conv1d(1, 32, kernel_size=2)
bn1 = nn.BatchNorm1d(32)
relu = nn.ReLU()
maxpool1 = nn.MaxPool1d(2)
conv2 = nn.Conv1d(32, 32, kernel_size=2)
maxpool2 = nn.MaxPool1d(2)

conv3 = nn.Conv1d(32, 64, kernel_size=2)
maxpool3 = nn.MaxPool1d(2)

conv4 = nn.Conv1d(64, 64, kernel_size=2)
maxpool4 = nn.MaxPool1d(2)

conv5 = nn.Conv1d(64, 128, kernel_size=2)
maxpool5 = nn.MaxPool1d(2)

conv6 = nn.Conv1d(128, 128, kernel_size=2)
maxpool6 = nn.MaxPool1d(2)

dropout1 = nn.Dropout(0.5)

conv7 = nn.Conv1d(128, 256, kernel_size=2)
maxpool7 = nn.MaxPool1d(2)

conv8 = nn.Conv1d(256, 256, kernel_size=2)
maxpool8 = nn.MaxPool1d(2)

dropout2 = nn.Dropout(0.5)

conv9 = nn.Conv1d(256, 512, kernel_size=2)
maxpool9 = nn.MaxPool1d(2)

dropout3 = nn.Dropout(0.5)

conv10 = nn.Conv1d(512, 512, kernel_size=2)
maxpool10 = nn.MaxPool1d(2)
        
conv11 = nn.Conv1d(512, 1024, kernel_size=1)
maxpool11 = nn.MaxPool1d(2)
        
dropout4 = nn.Dropout(0.5)


        # Fully Connected Blocks
flatten = nn.Flatten()
dense1 = nn.Linear(1024, 512)
        
batch_norm_dense1 = nn.BatchNorm1d(512)  # BatchNorm1d for Dense1
dropout5 = nn.Dropout(0.5)
        
dense2 = nn.Linear(512, 128)
        
batch_norm_dense2 = nn.BatchNorm1d(128)  # BatchNorm1d for Dense1
dropout6 = nn.Dropout(0.5)

dense3 = nn.Linear(128, 32)
batch_norm_dense3 = nn.BatchNorm1d(32)  # BatchNorm1d for Dense2

dense4 = nn.Linear(32, 1)

In [25]:
input_dummy=torch.rand(32,1,5000)

input_dummy = maxpool1(relu(bn1(conv1(input_dummy ))))
print(input_dummy.shape)
input_dummy= maxpool2(relu(conv2(input_dummy )))
print(input_dummy.shape)
input_dummy= maxpool3(relu(conv3(input_dummy )))
print(input_dummy.shape)
input_dummy= maxpool4(relu(conv4(input_dummy )))
print(input_dummy.shape)
input_dummy = maxpool5(relu(conv5(input_dummy )))
print(input_dummy.shape)
input_dummy =maxpool6(relu(conv6(input_dummy )))
print(input_dummy.shape)
input_dummy= dropout1(input_dummy )
print(input_dummy.shape)
input_dummy = maxpool7(relu(conv7(input_dummy )))
print(input_dummy.shape)
input_dummy= maxpool8(relu(conv8(input_dummy )))
print(input_dummy.shape)
input_dummy= dropout2(input_dummy )
print(input_dummy.shape)
input_dummy = maxpool9(relu(conv9(input_dummy )))
print(input_dummy.shape)
input_dummy = dropout3(input_dummy )
print(input_dummy.shape)
input_dummy = maxpool10(relu(conv10(input_dummy )))
print(input_dummy.shape)
input_dummy =maxpool11(relu(conv11(input_dummy )))
print(input_dummy.shape)
input_dummy =dropout4(input_dummy )
print(input_dummy.shape)


torch.Size([32, 32, 2499])
torch.Size([32, 32, 1249])
torch.Size([32, 64, 624])
torch.Size([32, 64, 311])
torch.Size([32, 128, 155])
torch.Size([32, 128, 77])
torch.Size([32, 128, 77])
torch.Size([32, 256, 38])
torch.Size([32, 256, 18])
torch.Size([32, 256, 18])
torch.Size([32, 512, 8])
torch.Size([32, 512, 8])
torch.Size([32, 512, 3])
torch.Size([32, 1024, 1])
torch.Size([32, 1024, 1])


# 테스트용 훈련코드 

In [31]:
# 모델을 GPU로 옮기기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Custom1DCNN2().to(device)

# 손실 함수 정의 (이진 크로스 엔트로피)
criterion = torch.nn.BCEWithLogitsLoss()

# 옵티마이저 선택 (Adam)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 및 검증 손실과 정확도를 기록할 리스트
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []
val_aurocs = []

# 학습 루프 설정
num_epochs = 10
for epoch in range(num_epochs):
    # 훈련 데이터 루프
    model.train()
    train_loss = 0.0
    train_preds = []
    train_targets = []
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        inputs, labels = inputs.to(device), labels.to(device)  # 데이터를 GPU로 이동
        optimizer.zero_grad()  # 그래디언트 초기화

        # 순전파 및 역전파
        outputs = model(inputs)
        loss = criterion(outputs.view(-1), labels.float())
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_preds.extend(torch.sigmoid(outputs).view(-1).cpu().detach().numpy())
        train_targets.extend(labels.cpu().numpy())

    train_loss /= len(train_loader)
    train_accuracy = accuracy_score(train_targets, np.round(train_preds))
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    # 검증 데이터 루프
    model.eval()
    val_preds = []
    val_targets = []
    val_loss = 0.0
    loss_check=[]
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs.view(-1), labels.float())
            val_loss += loss.item()
            val_preds.extend(torch.sigmoid(outputs).view(-1).cpu().detach().numpy())
            val_targets.extend(labels.cpu().numpy())

    val_loss /= len(val_loader)
    val_accuracy = accuracy_score(val_targets, np.round(val_preds))
    val_auroc = roc_auc_score(val_targets, val_preds)

    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    # 에포크의 결과 출력
    #print(f'Epoch [{epoch + 1}/{num_epochs}] - Train Loss: {train_loss:.4f} - Train Accuracy: {train_accuracy:.4f} - Validation Loss: {val_loss:.4f} - Validation Accuracy: {val_accuracy:.4f} - Validation AUROC: {val_auroc:.4f}')
    print(f'Epoch [{epoch + 1}/{num_epochs}] - Validation AUROC: {val_auroc:.4f}')
    val_aurocs.append(val_auroc)

Epoch 1/10 - Training: 100%|██████████| 545/545 [00:08<00:00, 62.12it/s]


Epoch [1/10] - Validation AUROC: 0.5758


Epoch 2/10 - Training: 100%|██████████| 545/545 [00:09<00:00, 59.25it/s]


Epoch [2/10] - Validation AUROC: 0.5848


Epoch 3/10 - Training: 100%|██████████| 545/545 [00:09<00:00, 58.71it/s]


...

Epoch [8/10] - Validation AUROC: 0.9622


Epoch 9/10 - Training: 100%|██████████| 545/545 [00:08<00:00, 61.19it/s]


Epoch [9/10] - Validation AUROC: 0.9730


Epoch 10/10 - Training: 100%|██████████| 545/545 [00:09<00:00, 60.03it/s]


Epoch [10/10] - Validation AUROC: 0.9747


In [32]:
# 사용자 정의 모듈을 임포트합니다.
from annie_data_prep import create_dataloaders, load_sph_data
from annie_cnn_model import SimpleCNN, Custom1DCNN, Custom1DCNNWithBatchNormAndDropout
from annie_cnnlstm_model import CNNLSTMModel



def objective(trial):
    ## 모델 시드 고정 
    seed = 42
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # 멀티 GPU를 사용하는 경우
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    

    # 옵튜나를 이용한 하이퍼파라미터 제안
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [64, 128])
    num_epochs = trial.suggest_int('num_epochs', 20, 30)  # Epoch 수를 동적으로 선택
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'SGD'])


    # 모델을 GPU로 옮기기
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = Custom1DCNN2().to(device)


    # 손실 함수 및 옵티마이저 설정
    criterion = nn.BCEWithLogitsLoss()
    if optimizer_name == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    elif optimizer_name == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    num_epochs = 100
    #batch_size = 128
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=10, factor=0.1, verbose=True)
    
    train_loader, val_loader, _ = create_dataloaders(batch_size)
    

    # 하이퍼파라미터 출력
    print("-"*40)
    print(f">>>>> Trial {trial.number+1} Start...<<<<<")
    print("-"*40)
    print(f"[Learning Rate] : {learning_rate}")
    print(f"[Batch Size] : {batch_size}")
    print(f"[Epochs] : {num_epochs}")  # 수정된 부분
    print(f"[Optimizer] : {optimizer}")
    print("-"*23)
    print(">>>>> Let's GO!!! <<<<<")
    print("-"*23)

    #print(f"Hidden Units: {hidden_units}")


    # 학습 및 검증 결과 기록을 위한 사전 선언
    model_info = {}

    # 학습 및 검증 과정에서의 손실과 정확도 기록
    train_losses = []
    train_accuracies = []
    train_aurocs = []  # 훈련 데이터 AUROC 기록을 위한 리스트
    train_auprcs = []  # 훈련 데이터 AUPRC 기록을 위한 리스트
    val_losses = []
    val_accuracies = []
    val_aurocs = []
    val_auprcs = []  # AUPRC 기록을 위한 리스트 추가

    
    #best_auroc = float('-inf')  # 최고 AUROC 기록을 위한 초기값 설정
    best_auprc = float('-inf')  # 최고 AUPRC 기록을 위한 초기값 설정
    best_auprc_info = None  # 최고 AUPRC 값을 가진 모델의 정보를 저장할 변수

    epochs_no_improve = 0
    early_stop = False
    patience = 20

    for epoch in range(num_epochs):
        # 훈련 루프
        model.train()
        train_loss = 0.0
        train_preds, train_targets = [], []

        for inputs, labels in tqdm(train_loader, desc=f"Trial {trial.number+1} - Epoch {epoch+1}/{num_epochs} - Training"):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()  # 그래디언트 초기화

            # 순전파 및 역전파
            outputs = model(inputs)
            loss = criterion(outputs.view(-1), labels.float())
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_preds.extend(torch.sigmoid(outputs).view(-1).cpu().detach().numpy())
            train_targets.extend(labels.cpu().numpy())

        train_loss /= len(train_loader)
        train_accuracy = accuracy_score(train_targets, np.round(train_preds))
        train_auroc = roc_auc_score(train_targets, train_preds)
        train_auprc = average_precision_score(train_targets, train_preds)
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        train_aurocs.append(train_auroc)
        train_auprcs.append(train_auprc)
        print(f">>> [Train] AUROC: {train_auroc:.4f} / AUPRC: {train_auprc:.4f}")



        # 검증 루프
        model.eval()
        val_loss, val_preds, val_targets = 0.0, [], []

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs.view(-1), labels.float())
                val_loss += loss.item()
                val_preds.extend(torch.sigmoid(outputs).view(-1).cpu().detach().numpy())
                val_targets.extend(labels.cpu().numpy())

        val_loss /= len(val_loader)
        val_accuracy = accuracy_score(val_targets, np.round(val_preds))
        val_auroc = roc_auc_score(val_targets, val_preds)
        val_auprc = average_precision_score(val_targets, val_preds)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)
        val_aurocs.append(val_auroc)
        val_auprcs.append(val_auprc)

        print(f">>> [Valid] AUROC: {val_auroc:.4f} / AUPRC: {val_auprc:.4f}")


        # 에포크 결과 기록
        epoch_info = {
            'train_loss': train_loss,
            'valid_loss': val_loss,
            'train_accuracy': train_accuracy,  
            'valid_accuracy': val_accuracy,      
            'train_auroc': train_auroc,
            'valid_auroc': val_auroc,
            'train_auprc': train_auprc,        
            'valid_auprc': val_auprc
        }
        model_info[epoch + 1] = epoch_info


        # 스케줄러 업데이트
        scheduler.step(val_auprc)

        # Early Stopping 체크 및 모델 저장
        if val_auprc > best_auprc:
            best_auprc = val_auprc
            epochs_no_improve = 0
            best_auprc_info = epoch_info  # 최고 AUPRC 값을 갱신할 때 정보 저장
            # 최고 성능 모델 저장
            torch.save(model.state_dict(), f'CL_trial_{trial.number+1}_best_model.pth')
            
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                print("Early stopping")
                break
    

        # 전체 학습 과정의 결과를 JSON 파일로 저장
        with open(f'CL_trial_{trial.number+1}_performance.json', 'w') as f:
            json.dump(model_info, f, indent=4)



    # 최고 AUPRC 값을 가진 모델의 정보 출력
    print("-" * 42)
    print(f"< Trial {trial.number+1}'s Best Performance>")
    if best_auprc_info is not None:
        items = list(best_auprc_info.items())
        for i, (key, value) in enumerate(items):
            print(f"[{key}]: {value:.4f}" + (" <- Pick It Up!" if i == len(items) - 1 else ""))

    return best_auprc

In [33]:
# 옵튜나 스터디 객체 생성 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

# 최적의 하이퍼파라미터 출력
best_trial = study.best_trial
best_model_path = f'trial_{best_trial.number+1}_best_model.pth'
best_performance_path = f'trial_{best_trial.number+1}_best_performance.json'

print(f"최고 성능을 보인 시도: {best_trial.number+1}")
print("최적의 하이퍼파라미터: {}".format(study.best_trial.params))

[I 2023-12-14 17:43:10,652] A new study created in memory with name: no-name-5f0eeb15-74a4-45e3-8f02-58bc5f718267


----------------------------------------
>>>>> Trial 1 Start...<<<<<
----------------------------------------
[Learning Rate] : 0.006329123088046764
[Batch Size] : 64
[Epochs] : 100
[Optimizer] : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.006329123088046764
    maximize: False
    weight_decay: 0
)
-----------------------
>>>>> Let's GO!!! <<<<<
-----------------------


Trial 1 - Epoch 1/100 - Training: 100%|██████████| 273/273 [00:06<00:00, 41.18it/s]


>>> [Train] AUROC: 0.5142 / AUPRC: 0.0734
>>> [Valid] AUROC: 0.4828 / AUPRC: 0.0777


Trial 1 - Epoch 2/100 - Training: 100%|██████████| 273/273 [00:06<00:00, 39.64it/s]


...

>>> [Train] AUROC: 0.9325 / AUPRC: 0.4326
>>> [Valid] AUROC: 0.9636 / AUPRC: 0.5605


Trial 2 - Epoch 7/100 - Training: 100%|██████████| 137/137 [00:05<00:00, 22.89it/s]


>>> [Train] AUROC: 0.9400 / AUPRC: 0.4708
>>> [Valid] AUROC: 0.9680 / AUPRC: 0.5857


Trial 2 - Epoch 8/100 - Training:  55%|█████▌    | 76/137 [00:03<00:02, 22.11it/s]
[W 2023-12-14 17:54:23,416] Trial 1 failed with parameters: {'learning_rate': 0.005523166782844678, 'batch_size': 128, 'num_epochs': 29, 'optimizer': 'Adam'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_238816/3750592422.py", line 100, in objective
    train_loss += loss.item()
KeyboardInterrupt
[W 2023-12-14 17:54:23,418] Trial 1 failed with value None.


KeyboardInterrupt: 